In [62]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import VotingClassifier
data = pd.read_csv('Pokemon.csv')

In [45]:
data = data.set_index('#')
data.head()
Y = data.pop('Type 1')

In [46]:
dummies = pd.get_dummies(data['Generation'])
#dummies2 = pd.get_dummies(data['Type 1'])
dummies3 = pd.get_dummies(data['Type 2'])

dummies1list = []
for x in dummies:
    word = 'Generation_' + str(x)
    dummies1list.append(word)
    
dummies.columns = dummies1list
step_1 = pd.concat([data, dummies], axis=1)  


#dummies2list = []
#for x in dummies2:
#    word = 'Type_1_' + x
#    dummies2list.append(word)
    
dummies3list = []
for x in dummies3:
    word = 'Type_2_' + x
    dummies3list.append(word)
#dummies2.columns = dummies2list
dummies3.columns = dummies3list
final_step = pd.concat([step_1,dummies3], axis = 1)
#data = pd.concat([final_step,dummies3], axis = 1)

#del data['Type 1']
del final_step['Type 2']
del final_step['Generation']
del final_step['Name']
final_step

#Converts the boolean column into a column of 0's and 1's
final_step.Legendary = final_step.Legendary.astype(int)

In [47]:
X_train, X_test, y_train, y_test = train_test_split( final_step, Y, test_size = 0.3, random_state = 100)

In [48]:
clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = 100,
                               max_depth=3, min_samples_leaf=5)
clf_gini.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=100,
            splitter='best')

In [49]:
pred = clf_gini.predict(X_test)

In [50]:
confusion_matrix(y_test,pred)

array([[ 1,  0,  1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  5,  0,  0,  0,  1,
         5],
       [ 0,  0,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  1,
         4],
       [ 0,  0,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4,  0,  0,  0,  0,
         3],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4,  0,  0,  0,  0,
        13],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4,  0,  0,  0,  0,
         4],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  7,  0,  0,  0,  0,
         0],
       [ 0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0,
         9],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,
         1],
       [ 0,  0,  1,  0,  0,  0,  0,  0,  2,  0,  0,  0,  1,  0,  0,  0,  0,
         1],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  0,  0,  4,  0,  0,  0,  1,
         9],
       [ 0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4,  0,  0,  0,  2,
         2],
       [ 0,  0,  0,  

In [42]:
accuracy_score(y_test,pred)

0.17083333333333334

In [51]:
#Gradient Boosting Classifier
GradBoost = GradientBoostingClassifier(n_estimators=350, learning_rate=0.1,
                                 max_depth=1, random_state=0)

GradBoost = GradBoost.fit(X_train,y_train)
predGradBoost = GradBoost.predict(X_test)


In [52]:
accuracy_score(y_test,predGradBoost)

0.23333333333333334

In [53]:
#Extremely Random Forest
Extratree = ExtraTreesClassifier(max_depth=None,min_samples_split=2)
Extratree = Extratree.fit(X_train,y_train)
predFinalExtRandomForest = Extratree.predict(X_test)


In [54]:
accuracy_score(y_test,predFinalExtRandomForest)

0.22500000000000001

In [55]:
#Adaptive Boosting
regr = AdaBoostClassifier(learning_rate = 1, n_estimators = 350)
regr = regr.fit(X_train,y_train)
adapred = regr.predict(X_test)


In [56]:
accuracy_score(y_test,adapred)

0.15833333333333333

In [63]:
finalensem = VotingClassifier(estimators=[('AdaBoost', regr), ('ExtraTrees', Extratree), ('GradientBoost', GradBoost)], voting='soft')

In [64]:
finalensem.fit(X_train,y_train)
ensempred = finalensem.predict(X_test)

In [65]:
accuracy_score(y_test,ensempred)

0.24166666666666667